Importation : 

In [9]:
from PIL import ImageTk ,ImageOps
import tkinter 
from tkinter.filedialog import *
import os
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import messagebox  
import cv2 
import colorsys
import PIL
from tkinter import * 
import IRM as irm

Fonctions :

In [10]:
def irmCodeTk(image_name,ctype):
    im=PIL.Image.open(image_name)
    image=np.array(im)
    shape=image.shape
    head=format(232,"08b")     #signature irm
    data_offset=40+16+32+32+4+4+16+16 
    head+=format(data_offset,"016b")
    file=""
    height=shape[0]    
    width=shape[1]      
    
    if(len(shape)==3):         #if len shape = 3  then kayn 3 plans or 4  
        nb_plan=shape[2]  
    elif(len(shape)==2): nb_plan=1     #if not then its gray
    if(im.mode=='L' and ctype==3) : ctype=1
    #add width , height , nb plans and compression type 
    head+=format(height,"032b")+format(width,"032b")+format(nb_plan,"04b")+format(ctype,"04b")
    
    
    if ctype==0:    #huffman
        huffman_code=irm.Huffman_codage(image)
        code=huffman_code[0]
        huff_entet=huffman_code[1]
    elif ctype==1:
        huffman_code=irm.Huffman_codage(irm.quant(image,8))
        code=huffman_code[0]
        huff_entet=huffman_code[1]
    elif ctype==2:
        huffman_code=irm.Huffman_codage(irm.rgb2hls(image))
        code=huffman_code[0]
        huff_entet=huffman_code[1]
    elif ctype==3:
        huffman_code=irm.Huffman_codage(irm.quant(np.array(irm.rgb2hls(image)*255,int),8))
        code=huffman_code[0]
        huff_entet=huffman_code[1]
    
    head+=format(huff_entet[3],"016b")+format(huff_entet[4],"016b")
    
    Total_file_size=160+len(code)
    
    head=head[:8]+format(Total_file_size,"032b")+head[8:]
    
    global new_path
    
    if(ctype==0) :  
        new_path = os.path.splitext(image_name)[0]+"_lossless.irm" 
    else: new_path = os.path.splitext(image_name)[0]+"_lossy.irm" 
    
    
    #head = [signature(1),Total_file_size(4),data_offset(2),height(4),width(4),nb_plan+ctype(1),]
    file=head+code
    
    irm.write_bin(file,new_path)
    
    spath["text"]=new_path
    ssave["text"]="Saved in : "
    
    lsize["text"]="Uncompressed Size : "+str(round(ori_size/1024,2))+" Ko \n\n IRM File Size : "+str(round(os.path.getsize(new_path)/1024,2))+" Ko "
    
    gain=(1-(os.path.getsize(new_path)/ori_size))*100
    
    lgain["text"]="Compression Gain : "+str(round(gain,2)) +"%"

def resizeTk(img1,width,height):
    #Resize the Image using resize method
    resized_img1= img1.resize((width,height), PIL.Image.ANTIALIAS)
    img11= ImageTk.PhotoImage(resized_img1)
    return img11
def ouvtk():
    global ori_path 
    global ori_size
    f=askopenfilename(title="choose a file",
                                         filetypes=[("Image files",(".png",".jpeg",".jpg",".tif",".bmp"))])
    im=PIL.Image.open(f) 
    displayed=resizeTk(im,500,300) 
    
    ori_path=f 
    
    show_img.config(image=displayed)
    show_img.photo_ref=displayed
    
    compress_btn["state"] = NORMAL

    
    h=(np.array(im).shape[0]) 
    w=(np.array(im).shape[1]) 
    
    if(im.mode=="L"):
        ori_size=(w*h)
        lsize["text"]="Uncompressed Size : "+str(round((ori_size)/1024,2))+" Ko "
    else : 
        ori_size=(w*h*3)
        lsize["text"]="Uncompressed Size : "+str(round(ori_size/1024,2))+" Ko "
    txt="Width : "+str(w)+"\n \n Height : "+str(h)+"\n \n Mode : "+ im.mode
    l3["text"]=txt 
    spath["text"]=""
    ssave["text"]=""
    lgain["text"]="Compression Gain : %"
    
def save():
    global save_path
    f=askopenfilename(title="choose a directory")
    save_path=f
    
    
def ouvtk2():
    global irm_path 
    f=askopenfilename(title="choose a file",
                                         filetypes=[("IRM files",(".irm"))])
    
    im=PIL.Image.fromarray(irm.irmDECO(f).astype(np.uint8))
    displayed1=resizeTk(im,500,300)
    show_img2.config(image=displayed1)
    show_img2.photo_ref=displayed1

<br>

In [14]:
#---------------------------------------------------interface------------------------------------------------------------------
master = tk.Tk(className='IRM Compressor')
screen_width = master.winfo_screenwidth()
screen_height = master.winfo_screenheight()

master.configure(bg='#f0efeb')
master.geometry(f"{screen_width-50}x680")
master.resizable(False, False)
master.iconphoto(False, tk.PhotoImage(file=r'C:\Users\mouad\OneDrive\Bureau\Proj Compression\images\logo.png'))

#Header 
header_frame=Frame(master ,bg='#f0efeb')
header_frame.grid(row =0, column = 0,columnspan=5)


header_lab=Label(header_frame,text = "IRM Compression",background="#f0efeb",font="Times 20 ",underline=True) 

header_lab.pack(pady=5) 


#Left Frame - Compression Frame
left_frame=LabelFrame(master ,background="#f0efeb",text="Compression",relief='sunken', borderwidth=2,
                      labelanchor = 'n')
left_frame.grid(row =1, column = 0,rowspan = 2,pady=5,padx=12,sticky="e") 


imimp=PIL.Image.open(r'C:\Users\mouad\OneDrive\Bureau\Proj Compression\images\IMPORT.png') 
imimp=resizeTk(imimp,166,25)

open_btn = Button(left_frame, image=imimp, command=ouvtk ,bg="#27a9f9",
                 height=25, width=166)


imcomp=PIL.Image.open(r'C:\Users\mouad\OneDrive\Bureau\Proj Compression\images\compress.png') 
imcomp=resizeTk(imcomp,107,25) 


compress_btn = Button(left_frame,image = imcomp ,command=lambda:irmCodeTk(ori_path,ctype.get()), bg="#27a9f9",
                     height=25, width=107  )
compress_btn["state"] = DISABLED

im1=PIL.Image.open(r'C:\Users\mouad\OneDrive\Bureau\Proj Compression\images\img.png') 
img=resizeTk(im1,500,300) 

show_img=Label(left_frame,image=img,borderwidth=2,relief="solid", bg="#f0efeb") 

open_btn.grid(row =2, column = 0 ,pady=10) 
compress_btn.grid(row =2, column = 1 ,pady=10) 
show_img.grid(row=3,column=0,columnspan=2,pady=10,padx=20)

#compression Type 
radb=Frame(left_frame ,background="#f0efeb")
radb.grid(row=0,column=0,columnspan=3)

ctype = IntVar()

ch = Label(radb, text="Choose Type :")
ch.grid(row=0,column=0)

R1 = Radiobutton(radb, text="Lossless ", variable=ctype, value=0)
R1.grid(row=0,column=1)

R2 = Radiobutton(radb, text="Lossy", variable=ctype, value=3)
R2.grid(row=0,column=2)


#Mid Frame - image caractéristiques
middle_frame=Frame(master,background="#f0efeb")
middle_frame.grid(row = 1, column = 1,rowspan = 2)
w=150 ; h=150 
txt="Width : "+"\n \n Height : "+"\n \n Mode : " 

l3 = Label(middle_frame, text =txt,background="#f0efeb",font="Times 13 italic bold",width=13)
l3.pack() 


#right frame - decompression Frame    
right_frame=LabelFrame(master,background="#f0efeb",relief='sunken', borderwidth=2,text="Decompression",
                       labelanchor = 'n')
right_frame.grid(row = 1, column = 2,rowspan = 2,padx=12) 


imdecomp=PIL.Image.open(r'C:\Users\mouad\OneDrive\Bureau\Proj Compression\images\importfile.png') 
imdecomp=resizeTk(imdecomp,166,25)
decompress_btn = Button(right_frame, image=imdecomp , command=ouvtk2 ,bg="#27a9f9",
                       height=25, width=166)

im2=PIL.Image.open(r'C:\Users\mouad\OneDrive\Bureau\Proj Compression\images\file.png') 
file=resizeTk(im2,500,300)
show_img2 = Label(right_frame, image=file,borderwidth=2,relief="solid" ,bg="#f0efeb")

decompress_btn.grid(row=1,column=0,columnspan=2,pady=25)
show_img2.grid(row=2,column=0,columnspan=2,pady=10,padx=20) 

#Bottom left Frame - sizes
bottom_left=Frame(master,background="#f0efeb") 
bottom_left.grid(row = 4, column = 0,columnspan = 1)

lsize=Label(bottom_left,text="" ,background="#f0efeb",font="Arial 12 bold",height=3)
lsize.pack(padx=50)

#Bottom Frame - Compression Gain 
bottom_frame=Frame(master,background="#f0efeb")
bottom_frame.grid(row = 5, column = 0,columnspan = 3)

lgain=Label(bottom_frame,text="Compression Gain : %",background="#f0efeb",font="Arial 15 bold")
lgain.pack(pady=20)



#save path 
save_path=Frame(master,background="#f0efeb") 
save_path.grid(row = 7, column = 0,columnspan = 3)

ssave=Label(save_path,text="",background="#f0efeb",font="Arial 15 bold")
ssave.pack(side=LEFT)

spath=Label(save_path,text="",background="#f0efeb",font="Arial 12 italic bold",fg="#00468B")
spath.pack(side=RIGHT)

master.mainloop() 